# Reflection Pattern Agent: Self-Critique → Improvement → Final Answer

This notebook demonstrates the Reflection Pattern:

1. The model produces an initial draft answer.
2. The model critiques that draft like a tough senior reviewer.
3. The model rewrites a final answer using that critique.

This is how we get output that is:
- higher quality
- lower hallucination risk
- executive ready
- auditable

We also optionally inject **evidence** (Wikipedia summaries via REST API) into the critique stage so the model can fact-check itself instead of blindly trusting its first draft.

In [ ]:
import os, requests, textwrap
from openai import OpenAI

# Before running:
# os.environ['OPENAI_API_KEY'] = 'sk-your-key'

client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))
WIKI_HEADERS = {'User-Agent': 'reflection-agent-notebook/1.0'}

def wikipedia_summary(topic: str) -> str:
    url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{topic}"
    r = requests.get(url, headers=WIKI_HEADERS, timeout=10)
    if r.status_code != 200:
        return f"[wiki] error {r.status_code}: {r.text[:200]}"
    d = r.json()
    title = d.get('title','')
    desc = d.get('description','')
    summary = d.get('extract','')
    return f"{title} — {desc}\n{summary}"

SYSTEM_DRAFT = """
You are an expert analyst.
Write the best possible answer to the user's question.
Be structured, factual, clear.
Do not mention any reflection steps. Just answer.
"""

SYSTEM_CRITIQUE = """
You are a ruthless senior reviewer.
Find weaknesses in the DRAFT ANSWER.
Call out:
1. Factual uncertainty / missing evidence
2. Missing context for execs
3. Overpromising or risky tone
4. Confusing structure
Then give clear instructions for how to fix it.
Do NOT rewrite the final answer here.
"""

SYSTEM_REVISE = """
You are an executive-grade communicator.
Rewrite a FINAL ANSWER using:
- The original question
- The original draft
- The critique feedback

Goals:
- Keep strong points
- Fix factual gaps
- Improve clarity and tone for a VP audience
- Be confident but never overpromise
Return only the final answer.
"""

def call_model(system_prompt: str, user_content: str):
    resp = client.responses.create(
        model='o4-mini',
        input=[
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': user_content},
        ],
    )
    # normalize
    if hasattr(resp, 'output_text'):
        return resp.output_text
    try:
        return resp.choices[0].message.content
    except Exception:
        if hasattr(resp, 'output'):
            if isinstance(resp.output, list):
                return '\n'.join(str(x) for x in resp.output)
            return str(resp.output)
        return str(resp)

def reflection_pipeline(user_question: str, evidence_topics=None):
    # Step 1: Draft
    draft = call_model(
        SYSTEM_DRAFT,
        f"USER QUESTION:\n{user_question}\n\nWrite the draft answer now."
    )

    # Step 2: Gather evidence (Wikipedia summaries)
    evidence_blocks = []
    if evidence_topics:
        for topic in evidence_topics:
            ev = wikipedia_summary(topic)
            evidence_blocks.append(f"[EVIDENCE for {topic}]\n{ev}")
    evidence_text = '\n\n'.join(evidence_blocks) if evidence_blocks else '(no external evidence provided)'

    critique_input = textwrap.dedent(f"""
    USER QUESTION:
    {user_question}

    DRAFT ANSWER:
    {draft}

    EVIDENCE:
    {evidence_text}

    Now critique the DRAFT ANSWER.
    """).strip()

    critique = call_model(SYSTEM_CRITIQUE, critique_input)

    # Step 3: Revise
    revise_input = textwrap.dedent(f"""
    USER QUESTION:
    {user_question}

    ORIGINAL DRAFT:
    {draft}

    CRITIQUE:
    {critique}

    Now produce the improved FINAL ANSWER.
    """).strip()

    final_answer = call_model(SYSTEM_REVISE, revise_input)

    return {
        'draft': draft,
        'critique': critique,
        'final': final_answer,
        'evidence_used': evidence_text,
    }


### Demo: Reflective brief for an exec visiting Hyderabad

In [ ]:
demo = reflection_pipeline(
    user_question=(
        'Explain why Hyderabad matters for technology and what a VP should know before traveling there.'
    ),
    evidence_topics=['Hyderabad']  # factual grounding: Wikipedia summary of Hyderabad
)
print('===== DRAFT ANSWER =====')
print(demo['draft'])
print('\n===== CRITIQUE =====')
print(demo['critique'])
print('\n===== FINAL ANSWER =====')
print(demo['final'])
print('\n===== EVIDENCE USED =====')
print(demo['evidence_used'])


### Why this matters
You now have:

- The *draft*: raw creativity and speed.
- The *critique*: safety, truth pressure, tone control.
- The *final answer*: what you'd actually send to leadership.

This is how you ship AI that doesn't just talk — it self-checks.
And because you log all three artifacts (draft, critique, final), you can defend what you shipped.